In [ ]:
#!pip install transformers

### Downloading Finbert

In [5]:
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [14]:
news = pd.read_csv("Data/News/news_scraped.csv")

news["title_sentiment_pos"] = None
news["title_sentiment_neg"] = None
news["title_sentiment_neu"] = None
news["content_sentiment_pos"] = None
news["content_sentiment_neg"] = None
news["content_sentiment_neu"] = None
news

,ticker,title,category,release_date,provider,url,article_id,content,release_datetime,title_sentiment_pos,title_sentiment_neg,title_sentiment_neu,content_sentiment_pos,content_sentiment_neg,content_sentiment_neu
0,AAPL,Berkshire Hathaway Earnings Highlights: Inflat...,news,2022-04-30,Investing.com,https://www.investing.com/news/stock-market-ne...,2814576,By Daniel Shvartsman Investing.com – Berkshire...,"Apr 30, 2022 12:02PM ET",None,None,None,None,None,None
1,AAPL,Berkshire bought $51 billion stock as Buffett ...,news,2022-04-30,Reuters,https://www.investing.com/news/stock-market-ne...,2814541,"By Jonathan Stempel OMAHA, Neb. (Reuters) - Wa...","Apr 30, 2022 10:31AM ET",None,None,None,None,None,None
2,AAPL,Wall Street closes sharply lower on Amazon slu...,news,2022-04-29,Reuters,https://www.investing.com/news/stock-market-ne...,2813996,By Bansari Mayur Kamdar and Noel Randewich (Re...,"Apr 29, 2022 07:05PM ET",None,None,None,None,None,None
3,AAPL,Stock Market Today: Dow Ends Brutal April With...,news,2022-04-29,Investing.com,https://www.investing.com/news/stock-market-ne...,2814348,By Yasin Ebrahim Investing.com – The Dow slump...,"Apr 29, 2022 04:17PM ET",None,None,None,None,None,None
4,AAPL,"S&P 500 Slumps as Amazon, Apple Trigger Sea of...",news,2022-04-29,Investing.com,https://www.investing.com/news/stock-market-ne...,2814320,By Yasin Ebrahim Investing.com – The S&P 500 f...,"Apr 29, 2022 02:58PM ET",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6523,GME,The Smart Way To Trade This Week’s Selloff,opinion,2022-01-07,Jani Ziedins,https://www.investing.com/analysis/the-smart-w...,200613941,Author's Publications The S&P 500 traded mostl...,"Jan 07, 2022 01:27AM ET",None,None,None,None,None,None
6524,GME,2021 Year In Review: Investing.com Comic Edition,opinion,2022-01-03,Investing.com,https://www.investing.com/analysis/2021-year-i...,200613143,Compiled by Jesse Cohen With just one trading ...,"Jan 03, 2022 01:35AM ET",None,None,None,None,None,None
6525,GME,"Bitcoin Closes 2021 At $46,300",opinion,2022-01-01,Crypto Briefing,https://www.investing.com/analysis/bitcoin-clo...,200613326,The flagship cryptocurrency is up only 58% ove...,"Jan 01, 2022 11:43PM ET",None,None,None,None,None,None
6526,GME,3 Sectors That Investors Can’t Ignore In 2022,opinion,2021-12-31,Chris Markoch,https://www.investing.com/analysis/3-sectors-t...,200613259,If you had GameStop (NYSE:GME) on your bingo c...,(Chris Markoch ),None,None,None,None,None,None


### Calculating sentiments

In [21]:
import re
import numpy as np

def predict(string):
  # s oriznutim
  string = str(string).strip() #.replace("\n","")
  string = re.sub(r'^By[^-.]*-', '', string)
  string = re.sub(r'http\S+', '', string)
  tokens = tokenizer.encode_plus(
      string, add_special_tokens=True,
      max_length=512, truncation=True,
      padding="max_length", return_tensors='pt'
  )
  outputs = model(**tokens)
  probs = torch.nn.functional.softmax(outputs[0], dim=-1)
  return probs[0].tolist()

In [23]:
# looping through data

for i, row in news.iterrows():
  save = False
  if np.isnan(row.title_sentiment_pos):
    save = True
    news.loc[i,"title_sentiment_pos"], \
    news.loc[i,"title_sentiment_neg"], \
    news.loc[i,"title_sentiment_neu"] = predict(row.title)

  if np.isnan(row.content_sentiment_pos):
    news.loc[i,"content_sentiment_pos"], \
    news.loc[i,"content_sentiment_neg"], \
    news.loc[i,"content_sentiment_neu"] = predict(row.content)

  if ((i % 100) == 0) and save:
    print('--- saving ' + str(i) + ' sentiments ('+ str(round(i/news.shape[0]*100,2))+'%) ---')
    name = "Data/News/news_scraped.csv"
    news.to_csv(name, index=False)

print('--- saving ' + str(i) + ' sentiments ('+ str(round(i/news.shape[0]*100,2))+'%) ---')
name = "Data/News/news_scraped.csv"
news.to_csv(name, index=False)


--- saving 100 sentiments (1.53%) ---
--- saving 200 sentiments (3.06%) ---
--- saving 300 sentiments (4.6%) ---
--- saving 400 sentiments (6.13%) ---
--- saving 500 sentiments (7.66%) ---
--- saving 600 sentiments (9.19%) ---
--- saving 700 sentiments (10.72%) ---
--- saving 800 sentiments (12.25%) ---
--- saving 900 sentiments (13.79%) ---
--- saving 1000 sentiments (15.32%) ---
--- saving 1100 sentiments (16.85%) ---
--- saving 1200 sentiments (18.38%) ---
--- saving 1300 sentiments (19.91%) ---
--- saving 1400 sentiments (21.45%) ---
--- saving 1500 sentiments (22.98%) ---
--- saving 1600 sentiments (24.51%) ---
--- saving 1700 sentiments (26.04%) ---
--- saving 1800 sentiments (27.57%) ---
--- saving 1900 sentiments (29.11%) ---
--- saving 2000 sentiments (30.64%) ---
--- saving 2100 sentiments (32.17%) ---
--- saving 2200 sentiments (33.7%) ---
--- saving 2300 sentiments (35.23%) ---
--- saving 2400 sentiments (36.76%) ---
--- saving 2500 sentiments (38.3%) ---
--- saving 2600 se

### Aggregating the sentiments

In [46]:
import pandas as pd
d = pd.read_csv("News/news_scraped.csv")
d["date"] = d["release_date"]
d["date"] =  pd.to_datetime(d["date"])

# attributing weekend articles to monday
d.loc[d["date"].dt.weekday==5,"date"] = d["date"] + pd.DateOffset(days=2)
d.loc[d["date"].dt.weekday==6,"date"] = d["date"] + pd.DateOffset(days=1)


d["sent_content"] = d["content_sentiment_pos"] - d["content_sentiment_neg"]
d["sent_title"] = d["title_sentiment_pos"] - d["title_sentiment_neg"]
d = d[(d["date"]>="2015-01-01")&(d["date"]<="2022-04-30")]

s = d.groupby(["ticker","date","category"])[["sent_content","sent_title"]].mean()
s["count"] = d.groupby(["ticker","date","category"])["content"].count()
s = s.reset_index()

In [47]:
news_data = s.loc[s["category"]=="news",["ticker","date","sent_content","sent_title","count"]]
news_data

,ticker,date,sent_content,sent_title,count
0,AAPL,2015-01-01,-0.836367,0.011386,1
2,AAPL,2015-01-05,-0.161674,0.066744,2
4,AAPL,2015-01-06,-0.239434,-0.575550,1
6,AAPL,2015-01-07,-0.471278,-0.679813,2
11,AAPL,2015-01-13,-0.946393,0.684244,1
...,...,...,...,...,...
23058,TSM,2022-01-12,0.117665,0.009454,1
23059,TSM,2022-01-13,0.541747,0.151298,4
23063,TSM,2022-01-21,-0.961155,0.025158,1
23069,TSM,2022-03-04,0.194201,0.801289,1


In [48]:
opinion_data = s.loc[s["category"]=="opinion",["ticker","date","sent_content","sent_title","count"]]
opinion_data

,ticker,date,sent_content,sent_title,count
1,AAPL,2015-01-02,0.048146,0.051361,1
3,AAPL,2015-01-05,-0.513625,-0.326702,5
5,AAPL,2015-01-06,0.207351,-0.255821,2
7,AAPL,2015-01-07,-0.374589,-0.198694,2
8,AAPL,2015-01-08,-0.367842,-0.398092,2
...,...,...,...,...,...
23071,TSM,2022-03-10,0.015716,-0.863739,1
23072,TSM,2022-04-05,-0.703144,-0.553057,1
23074,TSM,2022-04-11,-0.216539,0.005635,1
23075,TSM,2022-04-21,-0.954043,-0.722239,1


In [49]:
pd.merge(news_data,opinion_data, on=["ticker","date"],how='outer',suffixes=('_news', '_opinions')).to_csv("News/news_sentiments_agg.csv",index=False)